In [26]:
import pandas as pd
import re
from datetime import datetime, timedelta

In [27]:
df = pd.read_csv("customer_support_tickets.csv")

df.head()
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8469 entries, 0 to 8468
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Ticket ID                     8469 non-null   int64  
 1   Customer Name                 8469 non-null   object 
 2   Customer Email                8469 non-null   object 
 3   Customer Age                  8469 non-null   int64  
 4   Customer Gender               8469 non-null   object 
 5   Product Purchased             8469 non-null   object 
 6   Date of Purchase              8469 non-null   object 
 7   Ticket Type                   8469 non-null   object 
 8   Ticket Subject                8469 non-null   object 
 9   Ticket Description            8469 non-null   object 
 10  Ticket Status                 8469 non-null   object 
 11  Resolution                    2769 non-null   object 
 12  Ticket Priority               8469 non-null   object 
 13  Tic

,0
Ticket ID,0
Customer Name,0
Customer Email,0
Customer Age,0
Customer Gender,0
Product Purchased,0
Date of Purchase,0
Ticket Type,0
Ticket Subject,0
Ticket Description,0


In [28]:
df.rename(columns=lambda x: x.strip().lower().replace(' ', '_'), inplace=True)
df.columns
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

df['clean_message'] = df['ticket_description'].apply(clean_text)
df[['ticket_description', 'clean_message']].head()

,ticket_description,clean_message
0,I'm having an issue with the {product_purchase...,im having an issue with the productpurchased p...
1,I'm having an issue with the {product_purchase...,im having an issue with the productpurchased p...
2,I'm facing a problem with my {product_purchase...,im facing a problem with my productpurchased t...
3,I'm having an issue with the {product_purchase...,im having an issue with the productpurchased p...
4,I'm having an issue with the {product_purchase...,im having an issue with the productpurchased p...


In [29]:
def classify_issue(text):
    payment_keywords = ['payment', 'charged', 'billing', 'amount', 'transaction']
    login_keywords = ['login', 'password', 'account', 'signin']
    delivery_keywords = ['delivery', 'shipping', 'delay', 'late']
    refund_keywords = ['refund', 'return', 'cancel']
    bug_keywords = ['bug', 'error', 'crash', 'issue', 'problem']

    if any(word in text for word in payment_keywords):
        return 'PAYMENT'
    elif any(word in text for word in login_keywords):
        return 'LOGIN'
    elif any(word in text for word in delivery_keywords):
        return 'DELIVERY'
    elif any(word in text for word in refund_keywords):
        return 'REFUND'
    elif any(word in text for word in bug_keywords):
        return 'BUG'
    else:
        return 'GENERAL'

df['issue_type'] = df['clean_message'].apply(classify_issue)


In [30]:
def assign_priority(text):
    if 'urgent' in text or 'immediately' in text:
        return 'P0'
    elif 'refund' in text or 'payment' in text:
        return 'P1'
    elif 'login' in text or 'delivery' in text:
        return 'P2'
    else:
        return 'P3'

df['assigned_priority'] = df['clean_message'].apply(assign_priority)

In [31]:
def confidence_score(text):
    keywords = ['payment', 'login', 'delivery', 'refund', 'bug', 'error']
    count = sum(word in text for word in keywords)
    return round(min(count / 3, 1.0), 2)

df['classification_confidence'] = df['clean_message'].apply(confidence_score)

In [32]:
df[['ticket_description', 'issue_type', 'assigned_priority']].head()

,ticket_description,issue_type,assigned_priority
0,I'm having an issue with the {product_purchase...,PAYMENT,P3
1,I'm having an issue with the {product_purchase...,BUG,P3
2,I'm facing a problem with my {product_purchase...,BUG,P3
3,I'm having an issue with the {product_purchase...,BUG,P3
4,I'm having an issue with the {product_purchase...,DELIVERY,P2


In [33]:
sla_hours_map = {
    'P0': 2,    # Critical
    'P1': 6,    # High
    'P2': 12,   # Medium
    'P3': 24    # Low
}

In [34]:
df['sla_hours'] = df['assigned_priority'].map(sla_hours_map)

In [35]:
current_time = datetime.now()

df['due_time'] = df['sla_hours'].apply(
    lambda x: current_time + timedelta(hours=x)
)

In [36]:
# Identifying tickets that are close to SLA breach
df['sla_risk'] = df['due_time'].apply(
    lambda x: 'HIGH' if (x - datetime.now()).total_seconds() / 3600 <= 2 else 'LOW'
)


In [37]:
df[['assigned_priority', 'sla_hours', 'due_time']].head()

,assigned_priority,sla_hours,due_time
0,P3,24,2025-12-28 15:56:28.985759
1,P3,24,2025-12-28 15:56:28.985759
2,P3,24,2025-12-28 15:56:28.985759
3,P3,24,2025-12-28 15:56:28.985759
4,P2,12,2025-12-28 03:56:28.985759


In [38]:
issue_report = df.groupby('issue_type').size().reset_index(name='ticket_count')
issue_report

,issue_type,ticket_count
0,BUG,6175
1,DELIVERY,1047
2,GENERAL,69
3,LOGIN,755
4,PAYMENT,193
5,REFUND,230


In [39]:
priority_report = df.groupby('assigned_priority').size().reset_index(name='ticket_count')
priority_report

,assigned_priority,ticket_count
0,P0,197
1,P1,231
2,P2,266
3,P3,7775


In [40]:
final_columns = [
    'ticket_id',
    'ticket_description',
    'issue_type',
    'assigned_priority',
    'sla_hours',
    'due_time',
    'classification_confidence',
    'sla_risk'
]

final_df = df[final_columns]
final_df.to_csv('final_ticket_report.csv', index=False)